<a href="https://colab.research.google.com/github/harsh0522/ERA-V1/blob/main/S2%20-%20Assignment/Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from __future__ import print_function #__future__ module is a built-in module in Python that is used to inherit new feature
import torch #PyTorch is a Python package that provides two high-level features: 1- Tensor computation (like NumPy) with st
import torch.nn as nn #PyTorch provides the elegantly designed modules and classes torch.nn , torch.optim , Dataset , and DataLoader to help you create and train neural networks, 
                      #here (as nn) basically it is convention or short form use for torch.nn
import torch.nn.functional as F 
import torch.optim as optim
from torchvision import datasets, transforms #torchvision datasets for various videos and images, it is opensource

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input 30x30 OUtput 28x28 RF 3x3  
                                                    #nn.Conv2d(1, 32, 3, padding=1) here I/P image is using 3x3 matrix/kernel and has 32 channels hence 32 kernels so like this maybe
                                                    # 30x30x1 | (3x3)x32 | 28x28x32 

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input 30x30 OUtput 28x28 RF 5x5  ## 30x30x32 | (3x3)x64 | 28x28x64
        self.pool1 = nn.MaxPool2d(2, 2) #input 28x28 OUtput 14x14 RF 10x10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input 16x16 OUtput 14x14 RF 12x12 ##16x16x64 | (3x3)x128 | 14x14x128 
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #input 16x16 OUtput 14x14 RF 14x14 ##16x16x128 | (3x3)x256 | 14x14x256
        self.pool2 = nn.MaxPool2d(2, 2) #input 14x14 OUtput 7x7 RF 28x28 
        self.conv5 = nn.Conv2d(256, 512, 3) #input 7x7 OUtput 5x5 RF 30x30 ##7x7x256 | (3x3)x512 | 14x14x512
        self.conv6 = nn.Conv2d(512, 1024, 3) #input 5x5 OUtput 3x3 RF 32x32 ##5x5x512 | (3x3)x1024 | 3x3x1024
        self.conv7 = nn.Conv2d(1024, 10, 3) #input 3x3 OUtput 1x1 RF 34x34 ##3x3x1024 | (3x3)x10 ? why we are using only 10 kernels of 3x3 strange let me ask sir | 1x1x10 


    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) #Network is created with activation function relu
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) #Network is created with activation function relu
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        x = x.view(-1, 512)
        return F.log_softmax(x) #I think in this we have use log to fall values between 0 and 1 or may be log probabilities

In [26]:
!pip install torchsummary #to install any module in python we use pip
from torchsummary import summary #this module provide summary of a model that we made #summary(model, input_size=(1, 28, 28))
use_cuda = torch.cuda.is_available() #Returns a bool(means true or false) indicating if CUDA is currently available or not
device = torch.device("cuda" if use_cuda else "cpu") #if CUDA is not available use CPU
model = Net().to(device) #object will internally move all parameters and buffers to the device, dtype
summary(model, input_size=(1, 28, 28)) #provide summary of model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
Total params: 387,840
Trainable params: 387,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.34
Params size (MB): 1.48
Estimated Total Size (MB): 2.82
----------------------------------------------------------------


<ipython-input-25-bcd00704146e>:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) #I think in this we have use log to fall values between 0 and 1 or may be log probabilities


In [28]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [29]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [32]:
print('x_shape:',x.shape)

NameError: ignored

In [31]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/235 [00:00<?, ?it/s]<ipython-input-25-bcd00704146e>:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) #I think in this we have use log to fall values between 0 and 1 or may be log probabilities
  0%|          | 0/235 [00:00<?, ?it/s]


ValueError: ignored